In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error

import random
import math
import time
import datetime
import operator

In [3]:
plt.style.use('fivethirtyeight')
%matplotlib inline

# Import Datasets

In [4]:
confirmed_cases = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,33190,33384,33594,33908,34194,34366,34451,34455,34740,34994
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2964,3038,3106,3188,3278,3371,3454,3571,3667,3752
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,16404,16879,17348,17808,18242,18712,19195,19689,20216,20770
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,855,855,855,855,855,855,855,858,861,862
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,346,386,386,396,458,462,506,525,541,576


In [5]:
deaths_reported = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
deaths_reported.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,898,920,936,957,971,994,1010,1012,1048,1094
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,79,81,83,83,85,89,93,95,97,101
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,959,968,978,988,996,1004,1011,1018,1028,1040
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,52
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,19,21,21,22,23,23,26,26,26,27


In [6]:
recovered_cases = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
recovered_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,20103,20179,20700,20847,20882,21135,21216,21254,21454,22456
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1702,1744,1791,1832,1875,1881,1946,2014,2062,2091
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,11884,12094,12329,12637,13124,13124,13743,14019,14295,14792
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,800,800,802,802,803,803,803,803,803,803
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,108,117,117,117,117,118,118,118,118,124


In [7]:
# retrieved from daily reports
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/07-15-2020.csv')
latest_data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-16 04:44:59,34.223334,-82.461707,165,1,0,164.0,"Abbeville, South Carolina, US",672.728014,0.606061
1,22001.0,Acadia,Louisiana,US,2020-07-16 04:44:59,30.295065,-92.414197,1510,46,0,1464.0,"Acadia, Louisiana, US",2433.717463,3.046358
2,51001.0,Accomack,Virginia,US,2020-07-16 04:44:59,37.767072,-75.632346,1045,14,0,1031.0,"Accomack, Virginia, US",3233.692289,1.339713
3,16001.0,Ada,Idaho,US,2020-07-16 04:44:59,43.452658,-116.241552,4836,28,0,4808.0,"Ada, Idaho, US",1004.179930,0.578991
4,19001.0,Adair,Iowa,US,2020-07-16 04:44:59,41.330756,-94.471059,17,0,0,17.0,"Adair, Iowa, US",237.695749,0.000000


# Column Info

In [8]:
# fetch all columns from confirmed dataset
cols = confirmed_cases.keys()
cols

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '7/6/20', '7/7/20', '7/8/20', '7/9/20', '7/10/20', '7/11/20', '7/12/20',
       '7/13/20', '7/14/20', '7/15/20'],
      dtype='object', length=180)

In [9]:
# extract the date columns
confirmed = confirmed_cases.loc[:, cols[4] : cols[-1]]
deaths = deaths_reported.loc[:, cols[4] : cols[-1]]
recoveries = recovered_cases.loc[:, cols[4] : cols[-1]]

In [10]:
confirmed

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20
0,0,0,0,0,0,0,0,0,0,0,...,33190,33384,33594,33908,34194,34366,34451,34455,34740,34994
1,0,0,0,0,0,0,0,0,0,0,...,2964,3038,3106,3188,3278,3371,3454,3571,3667,3752
2,0,0,0,0,0,0,0,0,0,0,...,16404,16879,17348,17808,18242,18712,19195,19689,20216,20770
3,0,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,858,861,862
4,0,0,0,0,0,0,0,0,0,0,...,346,386,386,396,458,462,506,525,541,576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0,0,0,0,0,0,0,0,0,0,...,721,724,724,726,727,727,729,732,732,737
262,0,0,0,0,0,0,0,0,0,0,...,1284,1297,1318,1356,1380,1389,1465,1498,1516,1526
263,0,0,0,0,0,0,0,0,0,0,...,311,311,313,314,314,317,317,317,321,321
264,0,0,0,0,0,0,0,0,0,0,...,6262,6315,6364,6410,6457,6506,6552,6596,6643,6695


# Filter Countries


In [14]:
dates = confirmed.keys()

# across all countries
world_cases = []
total_deaths = []
mortality_rate = []
recovery_rate = []
total_recovered = []
total_active = []

# country specific cases
china_cases = []
india_cases = []
us_cases = []
brazil_cases = []
russia_cases = []

# country specific deaths
china_deaths = []
india_deaths = []
us_deaths = []
brazil_deaths = []
russia_deaths = []

# country specific recoveries
china_recoveries = []
india_recoveries = []
us_recoveries = []
brazil_recoveries = []
russia_recoveries = []

In [15]:
for i in dates:
    # fetch info from all countries based on this date
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # append information above to respective list    
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum - death_sum - recovered_sum)    
    mortality_rate.append(death_sum / confirmed_sum)
    recovery_rate.append(recovered_sum / confirmed_sum)
    
    # find the total confirmed cases for this date matching with the respective country    
    china_cases.append(confirmed_cases[confirmed_cases['Country/Region'] == 'China'][i].sum())
    india_cases.append(confirmed_cases[confirmed_cases['Country/Region'] == 'India'][i].sum())
    us_cases.append(confirmed_cases[confirmed_cases['Country/Region'] == 'US'][i].sum())
    brazil_cases.append(confirmed_cases[confirmed_cases['Country/Region'] == 'Brazil'][i].sum())
    russia_cases.append(confirmed_cases[confirmed_cases['Country/Region'] == 'Russia'][i].sum())
    
    # find the total reported deaths for this date matching with the respective country    
    china_deaths.append(deaths_reported[deaths_reported['Country/Region'] == 'China'][i].sum())
    india_deaths.append(deaths_reported[deaths_reported['Country/Region'] == 'India'][i].sum())
    us_deaths.append(deaths_reported[deaths_reported['Country/Region'] == 'US'][i].sum())
    brazil_deaths.append(deaths_reported[deaths_reported['Country/Region'] == 'Brazil'][i].sum())
    russia_deaths.append(deaths_reported[deaths_reported['Country/Region'] == 'Russia'][i].sum())
    
    # find the total recovered cases for this date matching with the respective country    
    china_recoveries.append(recovered_cases[recovered_cases['Country/Region'] == 'China'][i].sum())
    india_recoveries.append(recovered_cases[recovered_cases['Country/Region'] == 'India'][i].sum())
    us_recoveries.append(recovered_cases[recovered_cases['Country/Region'] == 'US'][i].sum())
    brazil_recoveries.append(recovered_cases[recovered_cases['Country/Region'] == 'Brazil'][i].sum())
    russia_recoveries.append(recovered_cases[recovered_cases['Country/Region'] == 'Russia'][i].sum())    

In [38]:
# print out the world cases for each date
number_list = [ f"{dates[i]} : {world_cases[i]}" for i in range(len(dates)) ]
number_list

['1/22/20 : 555',
 '1/23/20 : 654',
 '1/24/20 : 941',
 '1/25/20 : 1434',
 '1/26/20 : 2118',
 '1/27/20 : 2927',
 '1/28/20 : 5578',
 '1/29/20 : 6166',
 '1/30/20 : 8234',
 '1/31/20 : 9927',
 '2/1/20 : 12038',
 '2/2/20 : 16787',
 '2/3/20 : 19887',
 '2/4/20 : 23898',
 '2/5/20 : 27643',
 '2/6/20 : 30802',
 '2/7/20 : 34395',
 '2/8/20 : 37129',
 '2/9/20 : 40159',
 '2/10/20 : 42768',
 '2/11/20 : 44810',
 '2/12/20 : 45228',
 '2/13/20 : 60381',
 '2/14/20 : 66908',
 '2/15/20 : 69050',
 '2/16/20 : 71234',
 '2/17/20 : 73269',
 '2/18/20 : 75151',
 '2/19/20 : 75651',
 '2/20/20 : 76211',
 '2/21/20 : 76840',
 '2/22/20 : 78601',
 '2/23/20 : 78979',
 '2/24/20 : 79543',
 '2/25/20 : 80396',
 '2/26/20 : 81373',
 '2/27/20 : 82737',
 '2/28/20 : 84114',
 '2/29/20 : 86009',
 '3/1/20 : 88386',
 '3/2/20 : 90360',
 '3/3/20 : 92938',
 '3/4/20 : 95237',
 '3/5/20 : 98028',
 '3/6/20 : 101971',
 '3/7/20 : 106009',
 '3/8/20 : 109960',
 '3/9/20 : 113860',
 '3/10/20 : 118884',
 '3/11/20 : 126547',
 '3/12/20 : 132297',
 '3/

In [23]:
print(f"Confirmed sum: {confirmed_sum:,}")
print(f"Death sum: {death_sum:,}")
print(f"Recovered sum: {recovered_sum:,}")

Confirmed sum: 13,555,532
Death sum: 584,132
Recovered sum: 7,559,252


In [44]:
print(f"China cases: {china_cases}")

China cases: [548, 643, 920, 1406, 2075, 2877, 5509, 6087, 8141, 9802, 11891, 16630, 19716, 23707, 27440, 30587, 34110, 36814, 39829, 42354, 44386, 44759, 59895, 66358, 68413, 70513, 72434, 74211, 74619, 75077, 75550, 77001, 77022, 77241, 77754, 78166, 78600, 78928, 79356, 79932, 80136, 80261, 80386, 80537, 80690, 80770, 80823, 80860, 80887, 80921, 80932, 80945, 80977, 81003, 81033, 81058, 81102, 81156, 81250, 81305, 81435, 81498, 81591, 81661, 81782, 81897, 81999, 82122, 82198, 82279, 82361, 82432, 82511, 82543, 82602, 82665, 82718, 82809, 82883, 82941, 83014, 83134, 83213, 83306, 83356, 83403, 83760, 83787, 83805, 83817, 83853, 83868, 83884, 83899, 83909, 83912, 83918, 83940, 83944, 83956, 83959, 83959, 83964, 83966, 83968, 83970, 83975, 83976, 83990, 84010, 84011, 84018, 84024, 84029, 84038, 84044, 84054, 84063, 84063, 84063, 84063, 84081, 84084, 84095, 84102, 84103, 84106, 84106, 84123, 84128, 84146, 84154, 84161, 84160, 84171, 84177, 84186, 84191, 84195, 84198, 84209, 84216, 84228

# Daily Increase

In [45]:
def daily_increase(data):
    res = []
    for i in range(len(data)):
        res.append(data[0]) if i == 0 else res.append(data[i] - data[i - 1])
    return res

In [53]:
# find daily cases increase in the world and our countries
world_daily_cases_increase = daily_increase(world_cases)
china_daily_cases_increase = daily_increase(china_cases)
india_daily_cases_increase = daily_increase(india_cases)
us_daily_cases_increase = daily_increase(us_cases)
brazil_daily_cases_increase = daily_increase(brazil_cases)
russia_daily_cases_increase = daily_increase(russia_cases)

In [54]:
world_daily_cases_increase

[555,
 99,
 287,
 493,
 684,
 809,
 2651,
 588,
 2068,
 1693,
 2111,
 4749,
 3100,
 4011,
 3745,
 3159,
 3593,
 2734,
 3030,
 2609,
 2042,
 418,
 15153,
 6527,
 2142,
 2184,
 2035,
 1882,
 500,
 560,
 629,
 1761,
 378,
 564,
 853,
 977,
 1364,
 1377,
 1895,
 2377,
 1974,
 2578,
 2299,
 2791,
 3943,
 4038,
 3951,
 3900,
 5024,
 7663,
 5750,
 14407,
 11091,
 11146,
 14656,
 15837,
 19594,
 27921,
 29585,
 32308,
 33413,
 42165,
 41139,
 50817,
 62960,
 64400,
 67608,
 59577,
 64639,
 76416,
 76221,
 81401,
 83154,
 80376,
 71718,
 72672,
 76873,
 83775,
 86625,
 87658,
 76720,
 96782,
 69594,
 69927,
 80829,
 96712,
 87724,
 73346,
 80707,
 73339,
 75250,
 76806,
 89013,
 86940,
 84121,
 72947,
 68704,
 75299,
 77046,
 83681,
 87048,
 80587,
 77636,
 76077,
 79989,
 90022,
 89086,
 91517,
 83635,
 76246,
 76241,
 83599,
 84752,
 97105,
 96349,
 91813,
 78509,
 88284,
 96424,
 103028,
 106334,
 106900,
 98983,
 95326,
 86963,
 92522,
 102600,
 118089,
 121110,
 128645,
 106890,
 96251,
 1

In [55]:
china_daily_cases_increase

[548,
 95,
 277,
 486,
 669,
 802,
 2632,
 578,
 2054,
 1661,
 2089,
 4739,
 3086,
 3991,
 3733,
 3147,
 3523,
 2704,
 3015,
 2525,
 2032,
 373,
 15136,
 6463,
 2055,
 2100,
 1921,
 1777,
 408,
 458,
 473,
 1451,
 21,
 219,
 513,
 412,
 434,
 328,
 428,
 576,
 204,
 125,
 125,
 151,
 153,
 80,
 53,
 37,
 27,
 34,
 11,
 13,
 32,
 26,
 30,
 25,
 44,
 54,
 94,
 55,
 130,
 63,
 93,
 70,
 121,
 115,
 102,
 123,
 76,
 81,
 82,
 71,
 79,
 32,
 59,
 63,
 53,
 91,
 74,
 58,
 73,
 120,
 79,
 93,
 50,
 47,
 357,
 27,
 18,
 12,
 36,
 15,
 16,
 15,
 10,
 3,
 6,
 22,
 4,
 12,
 3,
 0,
 5,
 2,
 2,
 2,
 5,
 1,
 14,
 20,
 1,
 7,
 6,
 5,
 9,
 6,
 10,
 9,
 0,
 0,
 0,
 18,
 3,
 11,
 7,
 1,
 3,
 0,
 17,
 5,
 18,
 8,
 7,
 -1,
 11,
 6,
 9,
 5,
 4,
 3,
 11,
 7,
 12,
 58,
 49,
 43,
 44,
 36,
 36,
 0,
 59,
 19,
 52,
 29,
 20,
 28,
 24,
 18,
 14,
 23,
 5,
 31,
 14,
 8,
 19,
 14,
 18,
 28,
 33,
 42,
 0,
 79,
 46,
 0,
 109,
 20]

In [57]:
# find daily death increase in the world and our countries
world_daily_death_increase = daily_increase(total_deaths)
china_daily_death_increase = daily_increase(china_deaths)
india_daily_death_increase = daily_increase(india_deaths)
us_daily_death_increase = daily_increase(us_deaths)
brazil_daily_death_increase = daily_increase(brazil_deaths)
russia_daily_death_increase = daily_increase(russia_deaths)

In [60]:
# find daily recovery increase in the world and our countries
world_daily_recovery_increase = daily_increase(total_recovered)
china_daily_recovery_increase = daily_increase(china_recoveries)
india_daily_recovery_increase = daily_increase(india_recoveries)
us_daily_recovery_increase = daily_increase(us_recoveries)
brazil_daily_recovery_increase = daily_increase(brazil_recoveries)
russia_daily_recovery_increase = daily_increase(russia_recoveries)

# Visualize

In [64]:
# grab all the unique countries
unique_countries = list(latest_data['Country_Region'].unique())
unique_countries

['US',
 'Italy',
 'Brazil',
 'Russia',
 'Mexico',
 'Japan',
 'Canada',
 'Colombia',
 'Peru',
 'Spain',
 'India',
 'United Kingdom',
 'China',
 'Chile',
 'Netherlands',
 'Australia',
 'Pakistan',
 'Germany',
 'Sweden',
 'Ukraine',
 'Denmark',
 'France',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Central African Republic',
 'Chad',
 'Comoros',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Et

In [69]:
# find data for each country now
country_confirmed_cases = []
country_death_cases = []
country_active_cases = []
country_recovery_cases = []
country_mortality_rate = []
no_cases = []

for i in unique_countries:
    # grab the total number of cases for this country
    cases = latest_data[latest_data['Country_Region'] == i]['Confirmed'].sum()
    if cases > 0:
        country_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
        
# remove the countries which have no cases now
for i in no_cases:
    unique_countries.remove(i)

In [70]:
# sort the countries by the number of confirmed cases
unique_countries = [k for k, v in sorted(zip(unique_countries, country_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
unique_countries[0: 5]  # these 5 countries have the most cases

['US', 'Brazil', 'India', 'Russia', 'Peru']

In [72]:
for i in range(len(unique_countries)):
    country_confirmed_cases[i] = latest_data[latest_data['Country_Region'] == unique_countries[i]]['Confirmed'].sum()
    country_death_cases.append(latest_data[latest_data['Country_Region'] == unique_countries[i]]['Deaths'].sum())
    country_recovery_cases.append(latest_data[latest_data['Country_Region'] == unique_countries[i]]['Recovered'].sum())
    country_active_cases.append(country_confirmed_cases[i] - country_death_cases[i] - country_recovery_cases[i])
    country_mortality_rate.append(country_death_cases[i] / country_confirmed_cases[i])

In [75]:
# create a dataframe and show in a heatmap type table
country_df = pd.DataFrame({
    'Country Name': unique_countries,
    'Number of Confirmed Cases': country_confirmed_cases,
    'Number of Deaths': country_death_cases,
    'Number of Recoveries': country_recovery_cases,
    'Number of Active Cases': country_active_cases,
    'Mortality Rate': country_mortality_rate
})

country_df.style.background_gradient(cmap='Reds')

,Country Name,Number of Confirmed Cases,Number of Deaths,Number of Recoveries,Number of Active Cases,Mortality Rate
0,US,3497847,137407,1075882,2284558,0.039283
1,Brazil,1966748,75366,1350098,541284,0.038320
2,India,968857,24914,612768,331175,0.025715
3,Russia,745197,11753,522375,211069,0.015772
4,Peru,337751,12417,226400,98934,0.036764
5,Chile,321205,7186,292085,21934,0.022372
6,Mexico,317635,36906,252368,28361,0.116190
7,South Africa,311049,4453,160693,145903,0.014316
8,United Kingdom,293469,45138,1386,246945,0.153808
9,Iran,264561,13410,227561,23590,0.050688


In [76]:
# similar to unique country, do one for unique provinces
unique_provinces = list(latest_data['Province_State'].unique())
unique_provinces

['South Carolina',
 'Louisiana',
 'Virginia',
 'Idaho',
 'Iowa',
 'Kentucky',
 'Missouri',
 'Oklahoma',
 'Colorado',
 'Illinois',
 'Indiana',
 'Mississippi',
 'Nebraska',
 'Ohio',
 'Pennsylvania',
 'Washington',
 'Wisconsin',
 'Vermont',
 'Minnesota',
 'Florida',
 'North Carolina',
 'California',
 'New York',
 'Wyoming',
 'Michigan',
 'Alaska',
 'Maryland',
 'Kansas',
 'Tennessee',
 'Texas',
 'Maine',
 'Arizona',
 'Georgia',
 'Arkansas',
 'New Jersey',
 'South Dakota',
 'Alabama',
 'Oregon',
 'West Virginia',
 'North Dakota',
 'Massachusetts',
 'Utah',
 'Montana',
 'New Hampshire',
 'New Mexico',
 'Rhode Island',
 'Nevada',
 'District of Columbia',
 'Connecticut',
 'Hawaii',
 'Delaware',
 'Abruzzo',
 'Acre',
 'Adygea Republic',
 'Aguascalientes',
 'Aichi',
 'Akita',
 'Alagoas',
 'Alberta',
 'Altai Krai',
 'Altai Republic',
 'Amapa',
 'Amazonas',
 'Amur Oblast',
 'Ancash',
 'Andalusia',
 'Andaman and Nicobar Islands',
 'Andhra Pradesh',
 'Anguilla',
 'Anhui',
 'Antioquia',
 'Antofagasta

In [84]:
# find data for each province now
province_confirmed_cases = []
province_country = []
province_death_cases = []
province_active_cases = []
province_recovery_cases = []
province_mortality_rate = []
no_cases = []

for i in unique_provinces:
    # grab the total number of cases for this province
    cases = latest_data[latest_data['Province_State'] == i]['Confirmed'].sum()
    if cases > 0:
        province_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
        
# remove the countries which have no cases now
for i in no_cases:
    unique_provinces.remove(i)

In [85]:
# sort the provinces by the number of confirmed cases
unique_provinces = [k for k, v in sorted(zip(unique_provinces, province_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
unique_provinces[0: 5]  # these 5 provinces have the most cases

['New York', 'Sao Paulo', 'California', 'Florida', 'Texas']

In [86]:
for i in range(len(unique_provinces)):
    province_confirmed_cases[i] = latest_data[latest_data['Province_State'] == unique_provinces[i]]['Confirmed'].sum()
    province_country.append(l[latest_data['Province_State'] == unique_provinces[i]]['Country_Region'].unique()[0])
    province_death_cases.append(latest_data[latest_data['Province_State'] == unique_provinces[i]]['Deaths'].sum())
    province_recovery_cases.append(latest_data[latest_data['Province_State'] == unique_provinces[i]]['Recovered'].sum())
    province_active_cases.append(province_confirmed_cases[i] - province_death_cases[i] - province_recovery_cases[i])
    province_mortality_rate.append(province_death_cases[i] / province_confirmed_cases[i])

NameError: name 'lastest_data' is not defined

In [83]:
# create a dataframe and show in a heatmap type table
province_df = pd.DataFrame({
    'Province/State Name': unique_provinces,
    'Country': province_country,
    'Number of Confirmed Cases': province_confirmed_cases,
    'Number of Recoveries': province_recovery_cases,
    'Number of Active Cases': province_active_cases,
    'Mortality Rate': province_mortality_rate
})

province_df.style.background_gradient(cmap='Reds')

NameError: name 'province_country' is not defined